# 模型构造

Module类是nn模块里提供的一个模型构造类，是所有神经网络模块的基类，我们可以继承它来定义我们想要的模型。下面继承Module类构造本节开头提到的多层感知机。这里定义的MLP类重载了Module类的__init__函数和forward函数。它们分别用于创建模型参数和定义前向计算。前向计算也即正向传播。

In [2]:
import torch
from torch import nn

class MLP(nn.Module):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Module的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享”一节将介绍的模型参数params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256) # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)  # 输出层


    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [4]:
X = torch.rand(2, 784)
net = MLP()
print(net)
net(X).shape

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


torch.Size([2, 10])

- 可以通过继承Module类来构造模型。
- Sequential、ModuleList、ModuleDict类都继承自Module类。
- 与Sequential不同，ModuleList和ModuleDict并没有定义一个完整的网络，它们只是将不同的模块存放在一起，需要自己定义forward函数。
- 虽然Sequential等类可以使模型构造更加简单，但直接继承Module类可以极大地拓展模型构造的灵活性。

# 模型参数访问初始共享

In [8]:
import torch
from torch import nn
from torch.nn import init

net = nn.Sequential(nn.Linear(20, 100),
                    nn.ReLU(),
                    nn.Linear(100, 50),
                    nn.ReLU(), 
                    nn.Linear(50, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 20)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=20, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=1, bias=True)
)


***访问模型参数***

In [9]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([100, 20])
0.bias torch.Size([100])
2.weight torch.Size([50, 100])
2.bias torch.Size([50])
4.weight torch.Size([1, 50])
4.bias torch.Size([1])


可见返回的名字自动加上了层数的索引作为前缀。 我们再来访问net中单层的参数。对于使用Sequential类构造的神经网络，我们可以通过方括号[]来访问网络的任一层。索引0表示隐藏层为Sequential实例最先添加的层。

In [10]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([100, 20]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([100]) <class 'torch.nn.parameter.Parameter'>


In [ ]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad) # 反向传播前梯度为None
Y.backward()
print(weight_0.grad)

PyTorch的init模块里提供了多种预设的初始化方法。在下面的例子中，我们将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零。

In [ ]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

In [11]:
# 可以看到这就是一个inplace改变Tensor值的函数，而且这个过程是不记录梯度的。 类似的我们来实现一个自定义的初始化方法。在下面的例子里，我们令权重有一半概率初始化为0，有另一半概率初始化为[−10,−5][−10,−5]和[5,10][5,10]两个区间里均匀分布的随机数。

def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[ 9.2935,  5.0973, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
        [-0.0000, -8.6644,  9.3019,  ...,  0.0000,  6.2838,  9.6799],
        [ 5.5979,  7.6847,  5.3174,  ..., -0.0000, -0.0000, -5.9621],
        ...,
        [-9.3866, -0.0000,  8.1876,  ...,  7.3565,  5.8129, -0.0000],
        [ 0.0000, -5.6677, -8.7067,  ...,  5.1059, -0.0000, -9.9612],
        [-0.0000, -0.0000,  5.6455,  ...,  0.0000,  0.0000, -6.5547]])
2.weight tensor([[-7.4072, -8.5957, -0.0000,  ..., -7.0112,  0.0000, -0.0000],
        [-9.1385, -5.0109, -0.0000,  ...,  0.0000,  8.5964,  6.6367],
        [ 0.0000,  5.3679, -0.0000,  ...,  0.0000,  0.0000, -0.0000],
        ...,
        [-0.0000, -5.3281,  9.7068,  ..., -0.0000,  5.6696,  0.0000],
        [-0.0000, -0.0000,  5.7167,  ..., -9.6073, -5.0584, -0.0000],
        [-0.0000,  8.3538,  8.8340,  ...,  7.6395, -0.0000,  0.0000]])
4.weight tensor([[-8.4478,  7.5967,  7.3887, -8.6571,  9.8398,  0.0000, -6.3659, -0.0000,
          7.3684,  8.964

# 读取和存储

我们可以直接使用save函数和load函数分别存储和读取Tensor。save使用Python的pickle实用程序将对象进行序列化，然后将序列化的对象保存到disk，使用save可以保存各种对象,包括模型、张量和字典等。而load使用pickle unpickle工具将pickle的对象文件反序列化为内存。

In [12]:
import torch
from torch import nn

x = torch.ones(3)
torch.save(x, 'x.pt')

In [ ]:
x2 = torch.load('x.pt')
x2
y = torch.zeros(4)
torch.save([x, y], 'xy.pt')
xy_list = torch.load('xy.pt')
xy_list

In [ ]:
#存储并读取一个从字符串映射到Tensor的字典。

torch.save({'x': x, 'y': y}, 'xy_dict.pt')
xy = torch.load('xy_dict.pt')
xy

In [13]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = nn.Linear(3, 2)
        self.act = nn.ReLU()
        self.output = nn.Linear(2, 1)

    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

net = MLP()
net.state_dict()

OrderedDict([('hidden.weight', tensor([[ 0.2699,  0.0808,  0.2638],
                      [ 0.1859, -0.2168,  0.0757]])),
             ('hidden.bias', tensor([ 0.5552, -0.2102])),
             ('output.weight', tensor([[-0.0238,  0.6281]])),
             ('output.bias', tensor([0.1582]))])

In [14]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'params': [1384928621336, 1384928621480, 1384928621552, 1384928621624]}]}

PyTorch中保存和加载训练模型有两种常见的方法:

- 仅保存和加载模型参数(state_dict)；
- 保存和加载整个模型。

In [ ]:
torch.save(model.state_dict(), PATH) # 推荐的文件后缀名是pt或pth

#加载：
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))

In [ ]:
#保存：
torch.save(model, PATH)

#加载：
model = torch.load(PATH)

通过save函数和load函数可以很方便地读写Tensor。
通过save函数和load_state_dict函数可以很方便地读写模型的参数。

# GPU 計算

In [15]:
!nvidia-smi  # 对Linux/macOS用户有效

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
import torch
from torch import nn

torch.cuda.is_available() # 输出 True

False

In [ ]:
torch.cuda.device_count() # 输出 1

In [ ]:
#查看当前GPU索引号，索引号从0开始：
torch.cuda.current_device() # 输出 0

In [ ]:
#根据索引号查看GPU名字:
torch.cuda.get_device_name(0) # 输出 'GeForce GTX 1050'

In [ ]:
x = torch.tensor([1, 2, 3])
print(x)

#使用.cuda()可以将CPU上的Tensor转换（复制）到GPU上。
#如果有多块GPU，我们用.cuda(i)来表示第 ii 块GPU及相应
#的显存（ii从0开始）且cuda(0)和cuda()等价。
x = x.cuda(0)
x

In [ ]:
#我们可以通过Tensor的device属性来查看该Tensor所在的设备。
x.device

#我们可以直接在创建的时候就指定设备。
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.tensor([1, 2, 3], device=device)
# or
x = torch.tensor([1, 2, 3]).to(device)
x

In [ ]:
#可见模型在CPU上，将其转换到GPU上:
list(net.parameters())[0].device
net.cuda()
list(net.parameters())[0].device
#同样的，我么需要保证模型输入的Tensor和模型都在同一设备上，否则会报错。

x = torch.rand(2,3).cuda()
net(x)

***PyTorch要求计算的所有输入数据都在内存或同一块显卡的显存上***